In [1]:
query = "select * from smart-ruler-304409.product_attribute_module.app_review_keywords"
path = "app_review_keywords_csv"

In [2]:
import os
from pydatafabric.vault_utils import get_secrets

aws_info = get_secrets(mount_point="datafabric",path="aws/credentials/datafabric")

In [3]:
env = "dev" 
bucket = f"emart-datafabric-{env}"
destination = f"s3a://{bucket}/bigquery-db/{path}/op=put/"

In [4]:
from pydatafabric.ye import get_spark

spark = get_spark()
spark.conf.set("fs.s3a.fast.upload.buffer", "disk")
spark.conf.set("fs.s3a.buffer.dir", "/tmp")
spark.conf.set("fs.s3a.access.key", aws_info["aws_access_key_id"])
spark.conf.set("fs.s3a.secret.key", aws_info["aws_secret_access_key"])

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/09/30 13:05:30 INFO org.apache.spark.SparkEnv: Registering MapOutputTracker
22/09/30 13:05:30 INFO org.apache.spark.SparkEnv: Registering BlockManagerMaster
22/09/30 13:05:30 INFO org.apache.spark.SparkEnv: Registering BlockManagerMasterHeartbeat
22/09/30 13:05:30 INFO org.apache.spark.SparkEnv: Registering OutputCommitCoordinator


### CSV 포맷으로 저장

In [5]:
from pydatafabric.gcp import bq_to_df

df = bq_to_df(query, spark_session=spark)
df.write.mode("overwrite").csv(destination)

22/09/30 13:05:52 WARN org.apache.hadoop.fs.s3a.commit.AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.


In [7]:
!hdfs dfs -Dhadoop.security.credential.provider.path=jceks:///datafabric/credentials/aws.jceks -ls {destination}

2022-09-30 13:09:55,237 INFO impl.MetricsConfig: Loaded properties from hadoop-metrics2.properties
2022-09-30 13:09:55,291 INFO impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
2022-09-30 13:09:55,291 INFO impl.MetricsSystemImpl: s3a-file-system metrics system started
Found 2 items
-rw-rw-rw-   1 datafabric datafabric          0 2022-09-30 13:06 s3a://emart-datafabric-dev/bigquery-db/app_review_keywords_csv/op=put/_SUCCESS
-rw-rw-rw-   1 datafabric datafabric   49727095 2022-09-30 13:06 s3a://emart-datafabric-dev/bigquery-db/app_review_keywords_csv/op=put/part-00000-aef1e99b-9b7c-451e-afcd-8bb000c33c85-c000.csv
2022-09-30 13:09:56,969 INFO impl.MetricsSystemImpl: Stopping s3a-file-system metrics system...
2022-09-30 13:09:56,969 INFO impl.MetricsSystemImpl: s3a-file-system metrics system stopped.
2022-09-30 13:09:56,969 INFO impl.MetricsSystemImpl: s3a-file-system metrics system shutdown complete.
